In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from PIL import Image
import time
from ChessBoard import ChessBoard
import matplotlib.pyplot as plt
%matplotlib inline

模型路径

In [3]:
model_path = "./ass_net_3.0"

加载模型

In [4]:
sess = []
graph = []
for i in range(10):
    graph.append(tf.Graph())
    sess.append(tf.Session(graph = graph[i]))
    with graph[i].as_default():
        saver =tf.train.import_meta_graph(model_path + '/conv_ver_1.ckpt.meta')
        saver.restore(sess[i],tf.train.latest_checkpoint(model_path))

INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt
INFO:tensorflow:Restoring parameters from ./ass_net_3.0\conv_ver_1.ckpt


In [5]:
# for i in range(10):
#     print(graph[i])

提取输入输出接口

In [6]:
xs = []
w1 = []
b1=[]
w2=[]
b2=[]
w3=[]
b3=[]
y = []
for i in range(10):
    xs.append(graph[i].get_tensor_by_name('xs:0'))
    w1.append(graph[i].get_tensor_by_name('w1:0'))
    b1.append(graph[i].get_tensor_by_name('b1:0'))
    w2.append(graph[i].get_tensor_by_name('w2:0'))
    b2.append(graph[i].get_tensor_by_name('b2:0'))
    w3.append(graph[i].get_tensor_by_name('w3:0'))
    b3.append(graph[i].get_tensor_by_name('b3:0'))
    y.append(graph[i].get_tensor_by_name('y:0'))
    with graph[i].as_default():
        b1_r = tf.truncated_normal(shape=[3,3,1,8], stddev=0.02)
        b2_r = tf.truncated_normal(shape=[16], stddev=0.1)
        b1[i] = b1[i]+b1_r
        b2[i] = b2[i]+b2_r

网络变异函数

In [7]:
# 网络参数大致
# i 网络序号  sigma 变异程度
def variation(i,sigma=0.002):
    with graph[i].as_default():
        #卷积层1
        sess[i].run(tf.assign(w1[i],sess[i].run(w1[i])+tf.truncated_normal(shape=[3,3,1,8], stddev=sigma)))
        sess[i].run(tf.assign(b1[i],sess[i].run(b1[i])+tf.truncated_normal(shape=[8], stddev=0.1)))
        #卷积层2
        sess[i].run(tf.assign(w2[i],sess[i].run(w2[i])+tf.truncated_normal(shape=[3,3,8,16], stddev=sigma)))
        sess[i].run(tf.assign(b2[i],sess[i].run(b2[i])+tf.truncated_normal(shape=[16], stddev=0.1)))
        #全连接层
        sess[i].run(tf.assign(w3[i],sess[i].run(w3[i])+tf.truncated_normal(shape=[5*5*16, 1],stddev=sigma)))
        sess[i].run(tf.assign(b3[i],sess[i].run(b3[i])+tf.truncated_normal(shape=[1,1], stddev=0.1)))


查看网络参数的大致范围    为[10e1,10e-2]量级之间

In [9]:
print(sess[0].run(b1[0][0]))
print(sess[0].run(b3[0]))

[[[-0.04725843  0.4503509   0.01483895 -0.32782602 -0.17257856
    0.18062364  0.01153039 -0.09361976]]

 [[-0.05912223  0.44804642 -0.01609841 -0.32399443 -0.17313889
    0.15727869 -0.01517703 -0.1033792 ]]

 [[-0.06270165  0.48178342 -0.00884655 -0.31186587 -0.19540529
    0.18949273 -0.04235823 -0.12682837]]]
[[0.14847764]]


查看sigma对参数范围的影响

sigma = 0.1  1e-2 

sigma = 0.05 5*1e-4

sigma = 0.01 1e-4

In [51]:
with tf.Session() as s:
    z = tf.Variable(tf.truncated_normal(shape=[1,10000], stddev=0.05))
    s.run(tf.global_variables_initializer())
    print(s.run(tf.reduce_mean(z)))

0.00052546477


#### 查看网络运行速度
1000次网络前向传播，2.5秒

In [16]:
# 加载 测试集 标签
test_path = "C:/Users/王欣哲/Desktop/codes/人工智能基础/问题3 进化学习/dataset"
with open(test_path + "/test.csv") as f:
    test_data = np.loadtxt(f,delimiter=',') 
    test_len = len(test_data)
def get_test_batch(batch_size):
    idx = np.random.randint(1,test_len,size=batch_size)
    _chess=[]
    _label=[]
    for i in idx:
        _chess.append(test_data[i,0:225])
        _label.append(test_data[i,225])
    _chess = np.asfarray(_chess)
    _label = np.asfarray([_label]).reshape([batch_size,1])
    return _chess,_label/1000000

In [ ]:
t_start = time.time()

for j in range(100):
    batch = get_test_batch(1)
    for i in range(10):
        sess[i].run(y[i],feed_dict={xs[i]:batch[0]})
        
t_end = time.time()
print("time = ",t_end-t_start)
print("FPS = ",1.0/(t_end-t_start))